<a href="https://colab.research.google.com/github/luifo/IA-image-super-resolution-tensorflow/blob/test/flower_id_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Ruta en googledrive
PATH = "/content/drive/MyDrive/102flowers"


# New Section